# Génération de colonnes - Le problème de découpe binaire

Une compagnie de papier doit satisfaire les demandes pour des rouleaux de petite largeur qui doivent être découpés dans des grands rouleaux. Il faut déterminer les patrons de découpe des grands rouleaux de façon à satisfaire la demande tout en minimisant le nombre de grands rouleaux découpés.

Utilisez la méthode de génération de colonnes pour résoudre le problème avec des demandes (de une unité chacun) pour les petites largeurs 45m, 63m, 31m, 14m, 74m, 63m, 23m, 18m, 38m et 9m sachant que les grands rouleaux ont une largeur de 100m.

In [ ]:
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Données
Demandes = {1,2,3,4,5,6,7,8,9,10}
#Longueurs = {1:45, 2:63, 3:31, 4:14, 5:74, 6:63, 7:23, 8:18, 9:38, 10:9}
Longueurs = {1:45, 2:63, 3:31, 4:14, 5:74, 6:59, 7:23, 8:18, 9:38, 10:9}

# Fonction Problème maître restreint

In [4]:
def RMP(col):
    
    if col!=0:
        
        theta.append(master.continuous_var(0.0, 1.0,name='theta_'+str(len(theta))))
        #theta=master.continuous_var_list(7, 0.0, 1.0,name='theta')

        # On ajoute la variable dans toutes les contraintes concernées
        for i in col:
            ct[i-1].left_expr += theta[-1]

        master.obj+= theta[-1]
        master.minimize(master.obj)
        # Résolution
        master_sol = master.solve()

        master.print_solution()
        pi=master.dual_values([ct[i] for i in range(10)])
        print(pi)
        return(pi)
    else:
        master_sol = master.solve()

        master.print_solution()
        pi=master.dual_values([ct[i] for i in range(10)])
        print(pi)
        return(pi)
        
    

# Fonction Sous-problème

In [5]:
# Sous-problème

# Méthode exacte : un MIP
subproblem = Model(name='Sous-problème')

x = {(i): subproblem.binary_var(name='x_{0}'.format(i)) for i in Demandes}
def SP_MIP(pi) :
    


    #subproblem.maximize(x[1]*pi1 + x[2]*pi2 + x[3]*pi3 + x[4]*pi4 + x[5]*pi5 + x[6]*pi6 + x[7]*pi7 + x[8]*pi8 + x[9]*pi9 + x[10]*pi10 - 1)
    subproblem.minimize(1-subproblem.sum(x[i]*pi[i-1] for i in Demandes))

    subproblem.add_constraint(subproblem.sum(x[i]*Longueurs.get(i) for i in Demandes)<=100)
    sol = subproblem.solve()

    Z_SP=subproblem.solution.get_objective_value()
    l=list(subproblem.solution.get_value_dict(x).items())
    col=[l[i][0] for i in range(len(l)) if l[i][1]==1]
    if Z_SP >=0 :
        print("RMP optimal")
        return(0)
    else:
        print("nouvelle colonne : ",col)
        print("Z_SP =" ,Z_SP)
        return(col)
    

## Itération 1

In [6]:
# Problème maître restreint
# il faut trouver une solution réalisable.

master=Model(name='RMP')

theta=master.continuous_var_list(6, 0.0, 1.0,name='theta')

# contraintes : On doit prendre toutes les pièces
ct=[]
ct.append(master.add_constraint(theta[0]>=1))
ct.append(master.add_constraint(theta[1]>=1))
ct.append(master.add_constraint(theta[2]>=1))
ct.append(master.add_constraint(theta[2]>=1))
ct.append(master.add_constraint(theta[3]>=1))
ct.append(master.add_constraint(theta[4]>=1))
ct.append(master.add_constraint(theta[4]>=1))
ct.append(master.add_constraint(theta[5]>=1))
ct.append(master.add_constraint(theta[5]>=1))
ct.append(master.add_constraint(theta[5]>=1))


master.obj=master.sum(theta)
master.minimize(master.obj)
# Résolution
master_sol = master.solve()

courbe=[]
courbe.append(master.solution.get_objective_value()
)
master.print_solution()
    
# Sauvegarde des variables duales

pi=master.dual_values([ct[i] for i in range(10)])
print(pi)



objective: 6.000
  theta_0=1.000
  theta_1=1.000
  theta_2=1.000
  theta_3=1.000
  theta_4=1.000
  theta_5=1.000
[1.0, 1.0, 1.0, 0, 1.0, 1.0, 0, 1.0, 0, 0]


In [7]:
col=SP_MIP(pi)
print(col)

nouvelle colonne :  [1, 3, 8]
Z_SP = -2.0
[1, 3, 8]


# Autres Iterations

In [8]:
i=2
stop=False
while(stop==False):
    print("\n")
    print("---------------- itération ",i, "----------------")
    pi=RMP(col)
   
    col=SP_MIP(pi)
    i+=1
    if col==0:
        stop=True
        
    
    
    



---------------- itération  2 ----------------
objective: 6.000
  theta_0=1.000
  theta_1=1.000
  theta_2=1.000
  theta_3=1.000
  theta_4=1.000
  theta_5=1.000
[1.0, 1.0, 0, 1.0, 1.0, 1.0, 0, 0, 1.0, 0]
nouvelle colonne :  [1, 4, 9]
Z_SP = -2.0


---------------- itération  3 ----------------
objective: 5.500
  theta_1=1.000
  theta_2=0.500
  theta_3=1.000
  theta_4=1.000
  theta_5=1.000
  theta_6=0.500
  theta_7=0.500
[0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0, 0, 0, 1.0]
nouvelle colonne :  [4, 6, 10]
Z_SP = -1.5


---------------- itération  4 ----------------
objective: 5.333
  theta_1=1.000
  theta_2=0.333
  theta_3=1.000
  theta_4=1.000
  theta_5=0.667
  theta_6=0.667
  theta_7=0.333
  theta_8=0.333
[0.3333333333333333, 1.0, 0.6666666666666667, 0.3333333333333333, 1.0, 0, 1.0, 0, 0.3333333333333333, 0.6666666666666667]
nouvelle colonne :  [3, 4, 7, 10]
Z_SP = -1.666666666666667


---------------- itération  5 ----------------
objective: 5.000
  theta_1=1.000
  theta_2=0.200
  theta_3=1.0

La solution obtenue n'est pas entière.

# Branche 1

In [9]:
print(master.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: RMP

Minimize
 obj: theta_0 + theta_1 + theta_2 + theta_3 + theta_4 + theta_5 + theta_6
      + theta_7 + theta_8 + theta_9 + theta_10 + theta_11 + theta_12 + theta_13
      + theta_14 + theta_15 + theta_16 + theta_17 + theta_18 + theta_19
      + theta_20
Subject To
 c1: theta_0 + theta_6 + theta_7 + theta_17 + theta_20 >= 1
 c2: theta_1 + theta_10 + theta_11 + theta_12 >= 1
 c3: theta_2 + theta_6 + theta_9 + theta_10 + theta_16 + theta_19 + theta_20
      >= 1
 c4: theta_2 + theta_7 + theta_8 + theta_9 + theta_13 + theta_17 + theta_18 >= 
     1
 c5: theta_3 + theta_15 >= 1
 c6: theta_4 + theta_8 + theta_14 + theta_16 + theta_18 >= 1
 c7: theta_4 + theta_9 + theta_11 + theta_13 + theta_18 + theta_19 + theta_20
      >= 1
 c8: theta_5 + theta_6 + theta_12 + theta_13 + theta_15 >= 1
 c9: theta_5 + theta_7 + theta_13 + theta_14 + theta_17 + theta_19 >= 1
 c10: theta_5 + theta_8 + theta_9 + theta_11 + theta_16

In [10]:
ct.append(master.add_constraint(theta[4]==0))
ct.append(master.add_constraint(theta[18]==0))
pi = RMP(col)
ct_SP = subproblem.add_constraint(x[6]+x[7]<=1)
col = SP_MIP(pi)
print(master.export_as_lp_string())

objective: 4.000
  theta_7=1.000
  theta_11=1.000
  theta_15=1.000
  theta_16=1.000
[1.0, 1.0, 0, 0, 1.0, 1.0, 0, 0, 0, 0]
RMP optimal
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: RMP

Minimize
 obj: theta_0 + theta_1 + theta_2 + theta_3 + theta_4 + theta_5 + theta_6
      + theta_7 + theta_8 + theta_9 + theta_10 + theta_11 + theta_12 + theta_13
      + theta_14 + theta_15 + theta_16 + theta_17 + theta_18 + theta_19
      + theta_20
Subject To
 c1: theta_0 + theta_6 + theta_7 + theta_17 + theta_20 >= 1
 c2: theta_1 + theta_10 + theta_11 + theta_12 >= 1
 c3: theta_2 + theta_6 + theta_9 + theta_10 + theta_16 + theta_19 + theta_20
      >= 1
 c4: theta_2 + theta_7 + theta_8 + theta_9 + theta_13 + theta_17 + theta_18 >= 
     1
 c5: theta_3 + theta_15 >= 1
 c6: theta_4 + theta_8 + theta_14 + theta_16 + theta_18 >= 1
 c7: theta_4 + theta_9 + theta_11 + theta_13 + theta_18 + theta_19 + theta_20
      >= 1
 c8: theta_5 + theta_6 + theta_12 + theta_13 + theta_

# Branche 2

In [11]:
master.remove_constraint(ct[-1])
master.remove_constraint(ct[-1])

ct.append(master.add_constraint(theta[8]==0))
ct.append(master.add_constraint(theta[9]==0))
ct.append(master.add_constraint(theta[11]==0))
ct.append(master.add_constraint(theta[13]==0))
ct.append(master.add_constraint(theta[14]==0))
ct.append(master.add_constraint(theta[16]==0))
ct.append(master.add_constraint(theta[19]==0))
ct.append(master.add_constraint(theta[20]==0))

pi = RMP(col)
subproblem.remove_constraint(ct_SP)
subproblem.add_constraint(x[6]==x[7])
print(master.export_as_lp_string())

objective: 5.000
  theta_3=1.000
  theta_5=1.000
  theta_7=1.000
  theta_10=1.000
  theta_18=1.000
[1.0, 1.0, 0, 0, 1.0, 0, 1.0, 0, 0, 1.0]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: RMP

Minimize
 obj: theta_0 + theta_1 + theta_2 + theta_3 + theta_4 + theta_5 + theta_6
      + theta_7 + theta_8 + theta_9 + theta_10 + theta_11 + theta_12 + theta_13
      + theta_14 + theta_15 + theta_16 + theta_17 + theta_18 + theta_19
      + theta_20
Subject To
 c1: theta_0 + theta_6 + theta_7 + theta_17 + theta_20 >= 1
 c2: theta_1 + theta_10 + theta_11 + theta_12 >= 1
 c3: theta_2 + theta_6 + theta_9 + theta_10 + theta_16 + theta_19 + theta_20
      >= 1
 c4: theta_2 + theta_7 + theta_8 + theta_9 + theta_13 + theta_17 + theta_18 >= 
     1
 c5: theta_3 + theta_15 >= 1
 c6: theta_4 + theta_8 + theta_14 + theta_16 + theta_18 >= 1
 c7: theta_4 + theta_9 + theta_11 + theta_13 + theta_18 + theta_19 + theta_20
      >= 1
 c8: theta_5 + theta_6 + theta_12 + theta_13 + t